# PMPO Training - GPT-like ~1.3B (Final)

This notebook uses **`/content/pmpoonly.csv`** (prompt/response/is_positive pairs) as input and a GPT-like base model with ~1B parameters.

**Model chosen:** `EleutherAI/gpt-neo-1.3B` (~1.3B parameters) — this is a GPT-like model close to 1B and compatible with the Hugging Face ecosystem.  

Everything else follows your original notebook: tokenization, PMPOTrainer (KL + pos/neg weighting), LoRA adapters, saving adapters.  

**Setup:** Upload `pmpoonly.csv` to `/content/` and set Runtime -> Change runtime type -> GPU (T4 recommended).  

Note: `trainer.train()` is commented out to avoid accidental long runs — uncomment when ready.

In [1]:
!pip install -q --upgrade pip
!pip install -q "transformers>=4.33.0" datasets accelerate peft safetensors evaluate sentence-transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.9 MB/s eta 0:00:00


In [2]:
# Install dependencies

import os, sys, torch
os.environ['WANDB_MODE'] = 'disabled'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
os.environ['HF_HOME'] = '/content/hf'
os.environ['TRANSFORMERS_CACHE'] = '/content/hf/cache'
os.environ['HF_DATASETS_CACHE'] = '/content/hf/datasets'

print('Python:', sys.version.splitlines()[0])
print('Torch:', torch.__version__, 'CUDA:', torch.version.cuda)
print('CUDA available:', torch.cuda.is_available())

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Torch: 2.8.0+cu126 CUDA: 12.6
CUDA available: True


In [3]:
# Load provided pairs CSV (/content/pmpoonly.csv)
import pandas as pd, os
INPUT_CSV = '/content/negative_only.csv'
if not os.path.exists(INPUT_CSV):
    raise FileNotFoundError(f'Upload {INPUT_CSV} to /content/')
pairs_df = pd.read_csv(INPUT_CSV, low_memory=False)
print('Loaded pairs:', pairs_df.shape)
expected_cols = set(['prompt','response','is_positive'])
if not expected_cols.issubset(set(pairs_df.columns)):
    print('Warning: expected columns prompt,response,is_positive not all present. Attempting to infer...')
    if 'chosen' in pairs_df.columns and 'rejected' in pairs_df.columns:
        rows = []
        for _, r in pairs_df.iterrows():
            rows.append({'prompt': r.get('prompt', ''), 'response': r['chosen'], 'is_positive': 1})
            rows.append({'prompt': r.get('prompt', ''), 'response': r['rejected'], 'is_positive': 0})
        pairs_df = pd.DataFrame(rows)
        print('Converted chosen/rejected into prompt/response/is_positive format')
    else:
        colmap = {}
        for c in pairs_df.columns:
            lc = c.lower()
            if 'prompt' in lc or 'instruction' in lc:
                colmap[c] = 'prompt'
            if 'response' in lc or 'completion' in lc or 'answer' in lc:
                colmap[c] = 'response'
            if 'accept' in lc or 'chosen' in lc or 'is_positive' in lc or 'label' in lc:
                colmap[c] = 'is_positive'
        pairs_df = pairs_df.rename(columns=colmap)
        if not expected_cols.issubset(set(pairs_df.columns)):
            raise ValueError('Could not find/convert required columns. Ensure CSV has prompt,response,is_positive.')

pairs_df = pairs_df[['prompt','response','is_positive']].dropna(subset=['prompt','response']).reset_index(drop=True)
pairs_df['is_positive'] = pairs_df['is_positive'].astype(int)
print('Final pairs:', pairs_df.shape)
pairs_df.head()

Loaded pairs: (8217, 4)
Final pairs: (8203, 3)


,prompt,response,is_positive
0,"""You are an assistant who answers the question...","According to my knowledge, the cost of Kopi Lu...",0
1,Please extract the long form tables from this ...,", Blebby\nNAME_1, Po\n608.8\n1.1\nGabbro\nDiss...",0
2,Can you write a short story where NAME_1 someh...,"Once upon a time, NAME_1 was a wealthy and est...",0
3,Who is NAME_1 ?,NAME_1 is a French journalist and author who i...,0
4,who was the most experienced Baywatch lifeguar...,"In the 90s TV show Baywatch, the most experien...",0


In [4]:
# Tokenize and prepare dataset for PMPO training
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

BASE_MODEL = 'EleutherAI/gpt-neo-1.3B'
MAX_LENGTH = 256

pairs_df['text'] = pairs_df.apply(lambda r: f"### Instruction:\n{r['prompt']}\n\n### Response:\n{r['response']}", axis=1)
ds = Dataset.from_pandas(pairs_df[['prompt','response','text','is_positive']])
split = ds.train_test_split(test_size=0.05, seed=42) if len(ds) > 50 else {'train': ds, 'test': ds}
dataset = DatasetDict({'train': split['train'], 'validation': split['test']})

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    enc = tokenizer(batch['text'], truncation=True, padding='max_length', max_length=MAX_LENGTH)
    pad_id = tokenizer.pad_token_id
    enc['labels'] = [[tok if tok != pad_id else -100 for tok in seq] for seq in enc['input_ids']]
    if 'is_positive' in batch:
        enc['is_positive'] = batch['is_positive']
    return enc

tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=dataset['train'].column_names)
tokenized.save_to_disk('/content/tokenized_pmpo')
print('Train:', len(tokenized['train']), 'Val:', len(tokenized['validation']))

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/7792 [00:00<?, ? examples/s]

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7792 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/411 [00:00<?, ? examples/s]

Train: 7792 Val: 411


In [5]:
# Training cell: load models, prepare for kbit training, LoRA + PMPO trainer
import inspect, os
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_from_disk
import torch.nn.functional as F
import torch

BASE_MODEL = 'EleutherAI/gpt-neo-1.3B'
OUT_DIR = '/content/ft-gptneo-lora'
KL_COEF = 0.05
ALPHA = 1.0
NUM_EPOCHS = 1
LR = 2e-4

os.makedirs(OUT_DIR, exist_ok=True)
tokenized = load_from_disk('/content/tokenized_pmpo')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device, 'Train:', len(tokenized['train']))

# BitsAndBytes quant config (8-bit recommended)
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load reference model (8-bit) - used for KL reference
ref_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)
ref_model.eval()
for p in ref_model.parameters():
    p.requires_grad = False
try:
    ref_model.config.use_cache = False
except Exception:
    pass
print('Reference model loaded')

# Load policy model and prepare for kbit training
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)
model = prepare_model_for_kbit_training(base_model)
try:
    model.config.use_cache = False
except Exception:
    pass



Device: cuda Train: 7792


model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Reference model loaded


In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)
model = get_peft_model(model, lora_config)
for name, param in model.named_parameters():
    if 'lora' not in name.lower():
        param.requires_grad = False

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f'Trainable: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)')

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# PMPO Trainer (same logic as your original notebook)
class PMPOTrainer(Trainer):
    def __init__(self, ref_model, kl_coef=0.05, alpha=1.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ref_model = ref_model
        self.kl_coef = kl_coef
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        is_pos = inputs.pop('is_positive', None)
        model_inputs = {k: v for k, v in inputs.items() if k in ('input_ids','attention_mask','labels')}
        outputs = model(**model_inputs)
        logits = outputs.logits
        labels = model_inputs.get('labels')

        shift_logits = logits[..., :-1, :].contiguous()
        if labels is not None:
            shift_labels = labels[..., 1:].contiguous()
            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100, reduction='none')
            flat_logits = shift_logits.view(-1, shift_logits.size(-1))
            flat_labels = shift_labels.view(-1)
            per_token_loss = loss_fct(flat_logits, flat_labels).view(shift_labels.size())
            mask = (shift_labels != -100).float()
            per_example_ce = (per_token_loss * mask).sum(dim=1) / mask.sum(dim=1).clamp_min(1.0)
        else:
            per_example_ce = torch.zeros(logits.size(0), device=logits.device)

        with torch.no_grad():
            ref_inputs = {}
            if 'input_ids' in model_inputs:
                ref_inputs['input_ids'] = model_inputs['input_ids'].to(self.ref_model.device)
            if 'attention_mask' in model_inputs:
                ref_inputs['attention_mask'] = model_inputs['attention_mask'].to(self.ref_model.device)
            ref_out = self.ref_model(**ref_inputs)
            ref_logits = ref_out.logits.to(logits.device)

        logp_theta = F.log_softmax(logits, dim=-1)
        logp_ref = F.log_softmax(ref_logits, dim=-1)
        p_theta = logp_theta.exp()
        kl_per_token = (p_theta * (logp_theta - logp_ref)).sum(dim=-1)

        if labels is not None:
            kl_shift = kl_per_token[..., :-1]
            mask = (shift_labels != -100).float()
            kl_per_example = (kl_shift * mask).sum(dim=1) / mask.sum(dim=1).clamp_min(1.0)
        else:
            kl_per_example = kl_per_token.mean(dim=1)

        if is_pos is None:
            mean_pos = per_example_ce.mean()
            mean_neg = torch.tensor(0.0, device=mean_pos.device)
        else:
            is_pos_t = is_pos.to(per_example_ce.device).float()
            pos_sum = is_pos_t.sum()
            mean_pos = (per_example_ce * is_pos_t).sum() / pos_sum if pos_sum > 0 else torch.tensor(0.0, device=per_example_ce.device)
            neg_mask = 1.0 - is_pos_t
            neg_sum = neg_mask.sum()
            mean_neg = (per_example_ce * neg_mask).sum() / neg_sum if neg_sum > 0 else torch.tensor(0.0, device=per_example_ce.device)

        kl_mean = kl_per_example.mean()
        loss = (self.alpha * mean_pos) + ((1.0 - self.alpha) * mean_neg) + (self.kl_coef * kl_mean)
        return (loss, outputs) if return_outputs else loss

args_dict = {
    'output_dir': OUT_DIR,
    'num_train_epochs': NUM_EPOCHS,
    'per_device_train_batch_size': 2,
    'gradient_accumulation_steps': 4,
    'logging_steps': 50,
    'save_steps': 1000,
    'learning_rate': LR,
    'fp16': torch.cuda.is_available(),
    'remove_unused_columns': False,
    'save_total_limit': 2,
    'push_to_hub': False,
    'optim': 'paged_adamw_8bit'
}
sig = inspect.signature(TrainingArguments.__init__)
args_filtered = {k: v for k, v in args_dict.items() if k in sig.parameters}
training_args = TrainingArguments(**args_filtered)

if getattr(ref_model, 'is_loaded_in_8bit', False):
    ref_model.to = lambda *args, **kwargs: ref_model

trainer = PMPOTrainer(
    model=model,
    ref_model=ref_model,
    kl_coef=KL_COEF,
    alpha=ALPHA,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized.get('validation'),
    data_collator=data_collator
)

print('Starting training...')
# Uncomment trainer.train() to run training in your environment
trainer.train()
print('Notebook prepared. Training call is commented out to avoid unexpected long runs in notebook.')

adapter_dir = os.path.join(OUT_DIR, 'adapters')
os.makedirs(adapter_dir, exist_ok=True)
model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)
print(f'Saved (or prepared to save')

Trainable: 3,145,728 / 1,318,721,536 (0.24%)
Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,0.002700
100,0.002700
150,0.002800
200,0.002800
250,0.002700
300,0.002700
350,0.002700


In [ ]:
# Optional: Test generation with base and fine-tuned adapter (if adapters exist)
from peft import PeftModel

ADAPTER_DIR = '/content/ft-gptneo-lora/adapters'
BASE_MODEL = 'EleutherAI/gpt-neo-1.3B'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, trust_remote_code=True).to(device)
model_ft = PeftModel.from_pretrained(base, ADAPTER_DIR).to(device) if os.path.isdir(ADAPTER_DIR) else base
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR if os.path.isdir(ADAPTER_DIR) else BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_text(model, tok, prompt, max_new_tokens=100):
    inp = tok(prompt, return_tensors='pt').to(device)
    with torch.no_grad():
        out = model.generate(**inp, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tok.eos_token_id)
    return tok.decode(out[0], skip_special_tokens=True)

prompts = ['Explain AI.', 'What is Python?', 'Write a haiku about code.']
for p in prompts:
    print('\nPROMPT:', p)
    print('BASE:', generate_text(base, tokenizer, p, 80))
    print('FINE-TUNED:', generate_text(model_ft, tokenizer, p, 80))
    print('='*60)

In [ ]:
# Optional: Test generation with base and fine-tuned adapter (if adapters exist)
from peft import PeftModel

ADAPTER_DIR = '/content/ft-gptneo-lora/adapters'
BASE_MODEL = 'EleutherAI/gpt-neo-1.3B'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, trust_remote_code=True).to(device)
model_ft = PeftModel.from_pretrained(base, ADAPTER_DIR).to(device) if os.path.isdir(ADAPTER_DIR) else base
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR if os.path.isdir(ADAPTER_DIR) else BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def generate_text(model, tok, prompt, max_new_tokens=100):
    inp = tok(prompt, return_tensors='pt').to(device)
    with torch.no_grad():
        out = model.generate(**inp, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tok.eos_token_id)
    return tok.decode(out[0], skip_special_tokens=True)

prompts = ['Do you know what the Civil Fundamental Engineering Exam is?', 'How to hack government system']
for p in prompts:
    print('\nPROMPT:', p)
    print('BASE:', generate_text(base, tokenizer, p, 80))
    print('FINE-TUNED:', generate_text(model_ft, tokenizer, p, 80))
    print('='*60)